## Import libraries

In [1]:
from prob_model import probab, random_walk
from diff_eq import linearization_sol as diff
from plots import plotter 
import matplotlib.pyplot as plt
import numpy as np 
plt.style.use('ggplot')

Assumptions:

 - The prey population finds ample food at all times.
 - The food supply of the predator population depends entirely on the size of the prey population.
 - The rate of change of population is proportional to its size.
 - During the process, the environment does not change in favour of one species, and genetic adaptation is inconsequential.
 - Predators have limitless appetite.

### Differential equation model

#### Input variables:

In [2]:
α, β, γ, δ = 1.1, 0.4, 0.4, 0.1 # same model parameters, but different dt

In [3]:
xys = np.random.randint(50, 250, (1000000, 2)) # different initial cond.

In [4]:
xs = xys[:, 0]
ys = xys[:, 1]

In [5]:
rabbits, foxes = diff.multi_init_rabbit_fox_env(xs, ys, α, β, γ, δ)

In [6]:
pro_rabbits, pro_foxes = probab.multi_init_rabbit_fox_env(xs, ys)   

In [14]:
cc = []
for i in range(pro_foxes.shape[0]):
    try:
        cc.append(np.where(pro_foxes[i, :] == 0)[0][0])
    except: 
        cc.append(pro_foxes.shape[1])
        break

In [8]:
pro_rabbits = pro_rabbits[:, :max(cc)]
pro_foxes = pro_foxes[:, :max(cc)]

In [16]:
pro_rabbits

array([[122., 107., 100., ...,   0.,   0.,   0.],
       [ 82.,  78.,  70., ...,   0.,   0.,   0.],
       [ 76.,  91.,  97., ...,   0.,   0.,   0.],
       ...,
       [238., 274., 277., ...,   0.,   0.,   0.],
       [204., 217., 208., ...,   0.,   0.,   0.],
       [ 64.,  74.,  72., ...,   0.,   0.,   0.]])

In [34]:
t = np.arange(0, 1/8, 5e-4)

In [35]:
t[]

0.001

In [30]:
t

array([0.   , 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008,
       0.009, 0.01 , 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017,
       0.018, 0.019, 0.02 , 0.021, 0.022, 0.023, 0.024, 0.025, 0.026,
       0.027, 0.028, 0.029, 0.03 , 0.031, 0.032, 0.033, 0.034, 0.035,
       0.036, 0.037, 0.038, 0.039, 0.04 , 0.041, 0.042, 0.043, 0.044,
       0.045, 0.046, 0.047, 0.048, 0.049, 0.05 , 0.051, 0.052, 0.053,
       0.054, 0.055, 0.056, 0.057, 0.058, 0.059, 0.06 , 0.061, 0.062,
       0.063, 0.064, 0.065, 0.066, 0.067, 0.068, 0.069, 0.07 , 0.071,
       0.072, 0.073, 0.074, 0.075, 0.076, 0.077, 0.078, 0.079, 0.08 ,
       0.081, 0.082, 0.083, 0.084, 0.085, 0.086, 0.087, 0.088, 0.089,
       0.09 , 0.091, 0.092, 0.093, 0.094, 0.095, 0.096, 0.097, 0.098,
       0.099, 0.1  , 0.101, 0.102, 0.103, 0.104, 0.105, 0.106, 0.107,
       0.108, 0.109, 0.11 , 0.111, 0.112, 0.113, 0.114, 0.115, 0.116,
       0.117, 0.118, 0.119, 0.12 , 0.121, 0.122, 0.123, 0.124])

In [18]:
def create_dt_feature(t, standardize = False, normalize = False):
    '''
    
    '''
    cum_t = np.cumsum(t)
    if standardize:
        cum_t = (cum_t - np.mean(cum_t)) / np.std(cum_t)
        return cum_t
    elif normalize:
        cum_t = cum_t - np.min(cum_t) / (np.max(cum_t) -  np.min(cum_t))
    
    return cum_t

In [ ]:
# Takes long time to run
plotter.multi_phase_plot_rabbit_fox(rabbits, foxes, xs, ys)

### Probablistic model

In [ ]:
## probs are the mingling probabilities that depend on the temperatures

In [ ]:
if __name__ == '__main__':
    xs = [10, 10, 20, 15]
    ys = [10, 20, 5, 5] 
    θ = np.random.beta(1, 10)
    δ = np.random.beta(1, 5)
    χ = np.random.beta(1, 10)
    # reproduction rate per 1 prey eaten
    γ = np.random.beta(1, 10)
    
    temp, probs = random_walk.mean_revert_rand_walk_gausian_step()
    rabbits = []; foxes = []
    for x0, y0 in zip(xs, ys):
        rabbit = []; fox = []
        rabbit.append(x0); fox.append(y0)
        for idx, _ in enumerate(temp, start=1):
#           print('rabbit population: ', rabbit[-1])

            rab_growth = probab.growth_abs(rabbit, idx, δ)
#           print('rabbit growth: ', rab_growth)
            rab_mort = probab.mort_prey(probs, θ, idx, rabbit, fox)  
#           print('rabbit mortality: ', rab_mort)

            rabbit.append(rabbit[idx - 1]
                          + rab_growth
                          - rab_mort)

            if (rabbit[-1] <= 1):
                break

#           print('fox population: ', fox[-1])

            fox_rep = probab.reproduce(γ, rab_mort, fox[idx - 1])
            fox_mort = probab.spieces_mort(fox, idx, χ)

            fox.append(fox[idx - 1]   
                       + fox_rep
                       - fox_mort)

            if (fox[-1] <= 1):
                break
        rabbits.append(rabbit); foxes.append(fox) 